In [ ]:
# 3-6 레스토랑 리뷰 감성분류
import collections
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import re

from argparse import Namespace

Mounted at /content/drive


In [ ]:
# 만약 코랩에서 실행하는 경우 아래 코드를 실행하여 전처리된 라이트 버전의 데이터를 다운로드하세요.
!mkdir data
!wget https://git.io/JtRSq -O data/download.py
!wget https://git.io/JtRSO -O data/get-all-data.sh
!chmod 755 data/get-all-data.sh
%cd data
!./get-all-data.sh
%cd ..

--2024-03-12 09:59:45--  https://git.io/JtRSq
Resolving git.io (git.io)... 140.82.114.22
Connecting to git.io (git.io)|140.82.114.22|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py [following]
--2024-03-12 09:59:46--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_3/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘data/download.py’

data/download.py    100%[===================>]   1.54K  --.-KB/s    in 0s      

2024-03-12 09:59:47 (21.1 MB/s) - ‘data/download.py’ saved [1572/1572]

--2024-03-12 09:59:47--  https://git.io/JtRSO
Resolving git.io

In [16]:
# LITE 버전 데이터 전처리 수행
base_path = "/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter3_Basic_NeuralNet/"
args = Namespace(
    raw_train_dataset_csv= base_path + "data/yelp/raw_train.csv",
    raw_test_dataset_csv=base_path + "data/yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv= base_path + "data/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [17]:
# 원본 데이터를 읽습니다
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])
print(args.raw_train_dataset_csv)
print(train_reviews.head())

/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter3_Basic_NeuralNet/data/yelp/raw_train.csv
   rating                                             review
0       1  Unfortunately, the frustration of being Dr. Go...
1       2  Been going to Dr. Goldberg for over 10 years. ...
2       1  I don't know what Dr. Goldberg was like before...
3       1  I'm writing this review to give you a heads up...
4       2  All the food is great here. But the best thing...


In [24]:
# 리뷰 클래스 비율이 동일하도록 만듭니다
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    if _ < 5:
      # row를 딕셔너리로 --> column index: column value 형태로 변환
      print("row.to_dict():", row.to_dict())
      # 각 column index에 해당하는 value 반환
      print("row.rating:", row.rating)
    by_rating[row.rating].append(row.to_dict())

row.to_dict(): {'rating': 1, 'review': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}
row.rating: 1
row.to_dict(): {'rating': 2, 'review': "Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I hav

In [25]:
"""
a = collections.defaultdict(list)
a[0].append(1)
a[0].append(2)
a[1].append(3)
a[1].append(4)
print(a[0])
print(a[1])
print(a[2])
"""
## byrating = {1: [{dict1},{dict2}, ... , {dictn}], 2: [{dict1}, ...]}

'\na = collections.defaultdict(list)\na[0].append(1)\na[0].append(2)\na[1].append(3)\na[1].append(4)\nprint(a[0])\nprint(a[1])\nprint(a[2])\n'

In [31]:
review_subset = []

for _, item_list in sorted(by_rating.items()):
    n_total = len(item_list)
    #train에서 data일부를 분리
    n_subset = int(args.proportion_subset_of_train * n_total)
    #n_subset에 해당하는 데이터 수만큼 extend
    review_subset.extend(item_list[:n_subset])

for l in review_subset[:5]:
  print(l)
# review_subset 구조 -> [{}, {}, {}, {} .... {}] -> 총 전체 train_data의 10%에 해당.
review_subset = pd.DataFrame(review_subset)
print(review_subset.head())

{'rating': 1, 'review': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."}
{'rating': 1, 'review': "I don't know what Dr. Goldberg was like before  moving to Arizona, but let me tell you, STAY AWAY from this doctor and this office. I was going to Dr. Johnson before he left and Goldberg took over when Johnson left. He is not a caring doctor. He is only interested in the co-pay and having you come in fo

In [32]:
import pandas as pd

# 리스트 딕셔너리 형태의 예시 데이터 선언
data = [
     {'age': 18, 'english': 94, 'gender': 'male', 'math': 71, 'name': 'Douglas'},
     {'age': 18, 'english': 99, 'gender': 'male', 'math': 84, 'name': 'Darrell'},
     {'age': 16, 'english': 65, 'gender': 'female', 'math': 64, 'name': 'Karen'},
     {'age': 16, 'english': 93, 'gender': 'female', 'math': 74, 'name': 'Kathy'},
     {'age': 16, 'english': 87, 'gender': 'male', 'math': 93, 'name': 'David'}
]

# 데이터 프레임으로 전환
pd.DataFrame(data)
#ref: https://noanomal.tistory.com/290

,age,english,gender,math,name
0,18,94,male,71,Douglas
1,18,99,male,84,Darrell
2,16,65,female,64,Karen
3,16,93,female,74,Kathy
4,16,87,male,93,David


In [33]:
print(train_reviews.rating.value_counts())

# 고유 클래스
print(set(review_subset.rating))

1    280000
2    280000
Name: rating, dtype: int64
{1, 2}


In [36]:
# 훈련, 검증, 테스트를 만들기 위해 별점을 기준으로 나눕니다
by_rating = collections.defaultdict(list)

#LITE version --> review_subset을 최종 데이터 본으로
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())
##(참고) byrating = {1: [{dict1},{dict2}, ... , {dictn}], 2: [{dict1}, ...]}

# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed) # seed = 1337

n_train, n_val, n_test = 0, 0, 0
for _, item_list in sorted(by_rating.items()):
    # 배열을 무작위로 섞음
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

    # 데이터 포인터에 분할 속성을 추가합니다 (딕셔너리에 값추가) --> {"rating":1, "review":"ex-sentence..", "split":"train"}
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    # 최종 리스트에 추가합니다
    final_list.extend(item_list)

print(final_list[:5])
print(final_list[n_train:n_train + 5])
print(final_list[n_train+n_val:n_train+n_val+5])

[{'rating': 1, 'review': "Terrible place to work for I just heard a story of them find a girl over her biological father coming in there who she hadn't seen in 11 years she said hi to him which upset his wife and they left she finished the rest of her day working fine the next day when she went into work they fired over that situation. I for one and boycotting Texas Roadhouse because any place that could be that cruel to their staff does not deserve my business... Yelp wants me to give them a star but I don't believe they deserve it", 'split': 'train'}, {'rating': 1, 'review': '3 hours, 15 minutes-- total time for an extremely simple physical. Stay away unless you have hours to waste!!!', 'split': 'train'}, {'rating': 1, 'review': 'My less than stellar review is for service.   We waited 30 minutes for our meals to be delivered.  When we questioned the waiter, he was not helpful, so we asked to speak to the manager.  The manager did not even come to speak with us!   We were loyal neighb

In [37]:
# 분할 데이터를 데이터 프레임으로 만듭니다
final_reviews = pd.DataFrame(final_list)
final_reviews.split.value_counts()

train    39200
val       8400
test      8400
Name: split, dtype: int64

In [39]:
# 리뷰를 전처리합니다
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

#review열의 모든 항목에 preprocess_text 적용
final_reviews.review = final_reviews.review.apply(preprocess_text)

In [40]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)
final_reviews.head()

,rating,review,split
0,negative,terrible place to work for i just heard a stor...,train
1,negative,"hours , minutes total time for an extremely s...",train
2,negative,my less than stellar review is for service . w...,train
3,negative,i m granting one star because there s no way t...,train
4,negative,the food here is mediocre at best . i went aft...,train


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('review').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [43]:
print(args.output_munged_csv)
final_reviews.to_csv(args.output_munged_csv, index=False)

/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter3_Basic_NeuralNet/data/yelp/reviews_with_splits_lite.csv
